## A: Scenario Demonstration

### 1. Creating a server and three clients

### 2. Demonstrate how the coordinator works

### 3. Sending a message from one of the clients and replying back using private and broadcast messaging

### 4. Disconnecting a peer that is not the coordinator

### 5. Introducing a new client

### 6. Disconnecting the coordinator

## B: Implementation Inspection

### 1. Design patterns

### 2. Modularity/component design

### 3. Unit testing

### C: Additional points

### 1. Additional features

### 2. Subjects of interest